In [27]:
from socket import socket, AF_INET, SOCK_DGRAM
def send_goHome(goHome):
    ADDRESS = "192.168.2.102" # M5Stack address
    PORT = 8000
    s = socket(AF_INET, SOCK_DGRAM)
    s.sendto(str(goHome).encode(), (ADDRESS, PORT))
    s.close()
 

KeyboardInterrupt: 

In [37]:
send_goHome(0)

In [29]:
send_goHome(-1)

In [30]:
send_goHome(1)

In [2]:
import subprocess

print('start')
popen_obj = subprocess.Popen(['python3 hoge.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout_data, stderr_data = popen_obj.communicate()
print(popen_obj.returncode)
print('stdout_data:', stdout_data, ' stderr_data:', stderr_data)
print('end')

start


FileNotFoundError: [Errno 2] No such file or directory: 'python3 hoge.py': 'python3 hoge.py'